In [ ]:
!pip install diffusers==0.10.2 transformers scipy ftfy accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.1/503.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch import optim

from typing import Tuple, List
import numpy as np

import os
import tqdm
import json

from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms
from PIL import Image

from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler
from transformers import AutoProcessor, AutoModel

DEVICE = torch.device('cuda')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae") # latent -> image
text_encoder = AutoModel.from_pretrained("Bingsu/clip-vit-large-patch14-ko").text_model
tokenizer = AutoProcessor.from_pretrained("Bingsu/clip-vit-large-patch14-ko").tokenizer
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet") # -> latent
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=100)
device = "cuda"

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


...

merges.txt:   0%|          | 0.00/870k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

In [ ]:
class EmbededVector(nn.Module):
    def __init__(self):
        super(EmbededVector, self).__init__()

    def forward(self, text_embeded):
        out = text_embeded
        return out

In [ ]:
class LearnableVector(nn.Module):
    def __init__(self):
        super(LearnableVector, self).__init__()
        self.learnable_vector = nn.Parameter(torch.randn(4, 2, 77, 768))

    def forward(self, category):
        return self.learnable_vector[category]

In [ ]:
class Model(nn.Module):
    def __init__(self, eVector, lVector, vae, unet):
        super(Model, self).__init__()
        self.eVector = eVector
        self.lVector = lVector
        self.vae = vae
        self.unet = unet


    def forward(self, inputset):
        eVector_out = self.eVector(inputset)
        lVector_out = self.lVector(0)
        print(eVector_out.size())
        print(lVector_out.size())
        epl = torch.cat((eVector_out, lVector_out), dim=1)
        unet_out = self.unet(epl)
        vae_out = self.vae(unet_out)
        return vae_out

    def train(self, train_loader, criterion, optimizer, epoch, batch_size, device):

        for i in range(epoch):
            for batch_idx, (X, y) in enumerate(train_loader):
                X, y = X.to(device), y.to(device)
                #batch_noise = noise[batch_idx:batch_idx+len(X)]
                pred = self.forward(X)
                loss = criterion(pred, y)

                optimizer.zero_grad()
                loss.backward(retain_graph=True)
                optimizer.step()

                if batch_idx % 50 == 0:
                    print('Epoch: %d, [%5d/%5d] loss: %.6f' % (i+1, batch_idx*len(X), len(train_loader.dataset), loss))


In [ ]:
def preprocess_image(img) -> torch.Tensor:
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    img = transform(img)
    return img

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/AIproject/데이터 수집/small_data.xlsx'

df = pd.read_excel(path)
df.columns = ["index", "title", "author", "gender", "period", "theme", "contents", "is_textbook", "publisher", "image"]
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
texts = []
imgs = []

# iterrows()를 사용하여 각 행을 반복
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    # 'contents' 컬럼의 값을 text 리스트에 추가
    texts.append(row['contents'])

    # 'image' 컬럼의 값을 img_path로 저장하고 출력
    img_path = f"/content/drive/MyDrive/AIproject/데이터 수집/image{row['image'][1:]}"
    img = Image.open(img_path).convert('RGB')
    imgs.append(preprocess_image(img))


imgs = torch.stack(imgs, dim=0)

100%|██████████| 64/64 [00:02<00:00, 31.51it/s]


In [ ]:
def split_dataset(tokenizer, text_encoder, texts, imgs, device, saved: bool):
    # not use category now
    inputsets = []
    text_embededs = []
    for e in tqdm.tqdm(texts):
        encoded_text = tokenizer(e, padding="max_length", truncation=True, max_length=tokenizer.model_max_length, return_tensors="pt").to(device)
        text_embeded = text_encoder(encoded_text.input_ids.to(device))[0].squeeze(0)

        max_length = encoded_text.input_ids.shape[-1]
        uncond_input = tokenizer(
            [""] * len(e), padding="max_length", max_length=max_length, return_tensors="pt"
        )
        uncond_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]
        text_embeddings = torch.cat([uncond_embeddings, text_embeded])

        text_embededs.append(text_embeded)
    #print(tex)
    #inputsets = torch.stack(inputsets, dim=0)
    text_embededs = torch.stack(text_embededs, dim=0)

    k = int(len(text_embededs)*0.8)
    #train_dataset = TensorDataset(inputsets[:k], imgs[:k])
    #test_dataset = TensorDataset(inputsets[k:], imgs[k:])

    train_dataset = TensorDataset(text_embededs[:k], imgs[:k])
    test_dataset = TensorDataset(text_embededs[k:], imgs[k:])

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

    return train_loader, test_loader

In [ ]:
text_encoder.to(device)
train_loader, test_loader = split_dataset(tokenizer, text_encoder, texts, imgs, device, saved=False)

100%|██████████| 64/64 [00:01<00:00, 36.11it/s]


In [ ]:
model = Model(EmbededVector(), LearnableVector(), vae, unet)
model.to(device)
0

0

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
epoch = 500
batch_size=51
lr = 0.001
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
model.train(train_loader, criterion, optimizer, epoch, batch_size, DEVICE)

torch.Size([2, 77, 768])
torch.Size([2, 77, 768])


TypeError: ignored